In [107]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier 
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LinearRegression, LogisticRegression, LassoCV, RidgeCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.decomposition import PCA

%matplotlib inline

First, we have to decide how to deal with longitudinality. For now, we will ignore longitudinal data and treat each row as a unique patient. This means we need to take out any 'baseline' data.

In [13]:
df = pd.read_csv('ADNIMERGE.csv')
all_cols = list(df)
df.dtypes

# baseline data is indicated by '_bl' OR described as a baseline measurement in ADNIMERGE_DICT
important_cols = [i for i in all_cols[8:-3] if i[-2:] != 'bl']

In [14]:
important_cols = ['RID'] + important_cols

In [15]:
data = df[important_cols]

`AGE` is probably an important predictor for a diagnosis. However, ADNIMERGE_DICT says that `AGE` is the age of the patient at baseline. Because we are treating each row as a separate patient, we should use `Month_bl` (months from baseline) to calculate a 'new age' for each row. That is, a 65 year old who gets a second exam 12 months later will be treated as a different, 66 year old person.

In [16]:
data['EXACT_AGE'] = df.AGE + (df.Month_bl/12)

/Users/agasthyapradhan-shenoy/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [17]:
# now we need to one-hot encode the categorical predictors
# BUT since we will eventually do a classification with DX,
# we'll first convert those values and then one-hot encode
# things like marriage, etc

DX_class = []
for i in data.DX.values:
    if i == 'CN':
        DX_class.append(0)
    elif i == 'MCI':
        DX_class.append(1)
    elif i == 'Dementia':
        DX_class.append(2)
    else:
        DX_class.append(i) # for NaNs. will deal with missingness next.

# now drop the original DX column
data.drop('DX',axis=1,inplace=True)

# and replace with the new DX_class column
data['DX_class'] = DX_class

/Users/agasthyapradhan-shenoy/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/agasthyapradhan-shenoy/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [24]:
# get columns with categorical data
# by finding columns that have strings as entries
cat_cols = [i for i in data.columns if isinstance(data[i][0], str)]

In [26]:
# choosing to drop first because that was the 
# suggested best practice in HW3
data = pd.get_dummies(data, columns = cat_cols, drop_first = True)

In [28]:
data.shape

(13017, 55)

Now we need to deal with missingness.

In [29]:
# what percent of each column is missing?
def percent_in_each_col(df):
    percent_missing = [df[i].isnull().mean()*100 for i in list(df)]
    percent_missing = np.asarray(percent_missing)
    in_each = pd.DataFrame(percent_missing,df.columns,columns=["Percent"])
    print(in_each[in_each.Percent > 0])
    print("Average percent missing for all columns: ", in_each[in_each.Percent > 0].mean())


In [30]:
percent_in_each_col(data)

                         Percent
APOE4                   0.453253
FDG                    74.241377
PIB                    98.286856
AV45                   83.398633
CDRSB                  30.736729
ADAS11                 31.174618
ADAS13                 31.835292
MMSE                   29.522932
RAVLT_immediate        30.383345
RAVLT_learning         30.383345
RAVLT_forgetting       30.583084
RAVLT_perc_forgetting  31.120842
FAQ                    29.000538
MOCA                   60.052239
EcogPtMem              59.360836
EcogPtLang             59.483752
EcogPtVisspat          59.798725
EcogPtPlan             59.560575
EcogPtOrgan            60.382577
EcogPtDivatt           59.798725
EcogPtTotal            59.453023
EcogSPMem              59.099639
EcogSPLang             59.061228
EcogSPVisspat          59.829454
EcogSPPlan             59.545210
EcogSPOrgan            60.866559
EcogSPDivatt           60.121380
EcogSPTotal            59.099639
Ventricles             42.790197
Hippocampu

Some of the features have more than 50% of the data missing. This is because some tests were not used during certain protocols (ADNI1 vs ADNIGO, for example). Not sure exactly how to deal with that. Right now, we're just going to impute values for all the missing data, and see where that gets us.

In [32]:
# In lecture, Kevin mentioned that imputing using the median
# will often give you the best results

from sklearn.preprocessing import Imputer

imp = Imputer(strategy="median")
imputed = imp.fit_transform(data)
full_data = pd.DataFrame(imputed,columns = data.columns)

percent_in_each_col(full_data)

Empty DataFrame
Columns: [Percent]
Index: []
Average percent missing for all columns:  Percent   NaN
dtype: float64


In [34]:
full_data.head()

,RID,AGE,PTEDUCAT,APOE4,FDG,PIB,AV45,CDRSB,ADAS11,ADAS13,...,PTRACCAT_Hawaiian/Other PI,PTRACCAT_More than one,PTRACCAT_Unknown,PTRACCAT_White,PTMARRY_Married,PTMARRY_Never married,PTMARRY_Unknown,PTMARRY_Widowed,FLDSTRENG_3 Tesla MRI,FSVERSION_Cross-Sectional FreeSurfer (FreeSurfer Version 4.3)
0,2.0,74.3,16.0,0.0,1.36926,1.85,1.11467,0.0,10.67,18.67,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
1,3.0,81.3,18.0,1.0,1.09079,1.85,1.11467,4.5,22.00,31.00,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
2,3.0,81.3,18.0,1.0,1.06360,1.85,1.11467,6.0,19.00,30.00,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
3,3.0,81.3,18.0,1.0,1.10384,1.85,1.11467,3.5,24.00,35.00,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
4,3.0,81.3,18.0,1.0,1.03871,1.85,1.11467,8.0,25.67,37.67,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0


Now, we want to run a few different types of regressions on this 'full' data set using each cognitive test as a response variable. We'll create a matrix of R^2 values, and use that to decide which model should be used as our 'meta test'. **ALSO**: should we be standardizing the continuous variables??

In [141]:
def split_and_score(df, response_col_name, model):
    # returns TEST score based on model fitted to data with specified response var
    # response_col_name is a string
    # model is an unfitted sklearn model
    # would be nice to add a polynomial features option
    np.random.seed(9001)
    msk = np.random.rand(len(df)) < .75
    
    train = df[msk]
    test = df[~msk]
        
    predictors = [i for i in df.columns if i not in response_col_name]
    x = train[predictors]
    y = train[response_col_name]
    
    Xt = test[predictors]
    yt = test[response_col_name]        
    
    model.fit(x,y)
    test_score = model.score(Xt, yt)
    return test_score 

In [143]:
def make_score_df(df,response_vars,model_dict):
    # returns a dataframes of TEST r^2 values
    # response_vars is a list of strings
    # model_dict is a dictionary of unfitted sklearn models
    
    keys = list(model_dict.keys())
    scores = np.zeros((len(response_vars),len(keys)))
    
    for col, model in enumerate(keys):
        for row, response in enumerate(response_vars):
            scores[row,col] = split_and_score(df,response,model_dict[model])
   
    score_df = pd.DataFrame(scores,columns=[keys],index=response_vars)

    return score_df
    

In [130]:
cog_tests = ['MMSE','RAVLT_perc_forgetting','FAQ']

knn = KNeighborsRegressor(n_neighbors = 5)
lin_reg = LinearRegression(fit_intercept = True)
ridge = RidgeCV()
lasso = LassoCV()

models = {}
models['KNN'] = knn
models['LinearRegression'] = lin_reg
models['Ridge'] = ridge
models['Lasso'] = lasso

In [139]:
make_score_df(full_data,cog_tests,models)

,KNN,LinearRegression,Ridge,Lasso
MMSE,0.261538,0.684882,0.685186,0.190325
RAVLT_perc_forgetting,0.009418,0.325548,0.325082,0.036030
FAQ,0.283943,0.782938,0.779865,0.187039
